# Training a TFNO on Darcy-Flow

We use the example found in [`auto_examples_jupyter.zip`](https://neuraloperator.github.io/neuraloperator/dev/auto_examples/index.html) and so we'll demonstrate how to use the small Darcy-Flow example to train a Tensorized Fourier-Neural Operator.

In [1]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import TFNO
from neuralop import Trainer
from neuralop.training import OutputEncoderCallback, CheckpointCallback
from neuralop.datasets import load_darcy_flow_small
from neuralop.utils import count_model_params
from neuralop import LpLoss, H1Loss

# From running .to on a model, neuralop.models.fno.TFNO,
# RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: gpu

device = 'cuda'

/home/propdev/Prop/neuraloperator/neuralop/models/base_model.py:21: UserWarning: Creating a subclass of BaseModel FNO1d with no name, initializing with FNO1d.
  warnings.warn(f'Creating a subclass of BaseModel {cls.__name__} with no name, initializing with {cls.__name__}.')
/home/propdev/Prop/neuraloperator/neuralop/models/base_model.py:21: UserWarning: Creating a subclass of BaseModel FNO2d with no name, initializing with FNO2d.
  warnings.warn(f'Creating a subclass of BaseModel {cls.__name__} with no name, initializing with {cls.__name__}.')
/home/propdev/Prop/neuraloperator/neuralop/models/base_model.py:21: UserWarning: Creating a subclass of BaseModel FNO3d with no name, initializing with FNO3d.
  warnings.warn(f'Creating a subclass of BaseModel {cls.__name__} with no name, initializing with {cls.__name__}.')
/home/propdev/Prop/neuraloperator/neuralop/models/base_model.py:21: UserWarning: Creating a subclass of BaseModel TFNO with no name, initializing with TFNO.
  warnings.warn(f'

In [2]:
train_loader, test_loaders, output_encoder = load_darcy_flow_small(
    n_train=1000, batch_size=32,
    test_resolutions=[16, 32], n_tests=[100, 50],
    test_batch_sizes=[32, 32])

UnitGaussianNormalizer init on 1000, reducing over [0, 1, 2, 3], samples of shape [1, 16, 16].
   Mean and std of shape torch.Size([1, 1, 1]), eps=1e-05
Loading test db at resolution 32 with 50 samples and batch-size=32


In [3]:
# <class 'torch.utils.data.dataloader.DataLoader'>
print(type(train_loader))
# <class 'dict'>
print(type(test_loaders))
# 2
print(len(test_loaders))
# dict_keys([16, 32])
print(test_loaders.keys())
# <class 'neuralop.utils.UnitGaussianNormalizer'>
print(type(output_encoder))

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'dict'>
2
dict_keys([16, 32])
<class 'neuralop.utils.UnitGaussianNormalizer'>


We create a tensorized FNO model

In [4]:
model = TFNO(n_modes=(16, 16), hidden_channels=32, projection_channels=64, factorization='tucker', rank=0.42)
print("--------------------- model before to type: ", type(model))
model = model.to(device)
print("--------------------- model after to type: ", type(model))

n_params = count_model_params(model)
print(type(n_params))
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()

Keyword argument in_channels not specified for model TFNO, using default=3.
Keyword argument out_channels not specified for model TFNO, using default=1.
Keyword argument lifting_channels not specified for model TFNO, using default=256.
Keyword argument n_layers not specified for model TFNO, using default=4.
Keyword argument output_scaling_factor not specified for model TFNO, using default=None.
Keyword argument max_n_modes not specified for model TFNO, using default=None.
Keyword argument fno_block_precision not specified for model TFNO, using default=full.
Keyword argument use_mlp not specified for model TFNO, using default=False.
Keyword argument mlp_dropout not specified for model TFNO, using default=0.
Keyword argument mlp_expansion not specified for model TFNO, using default=0.5.
Keyword argument non_linearity not specified for model TFNO, using default=<built-in function gelu>.
Keyword argument stabilizer not specified for model TFNO, using default=None.
Keyword argument norm not

Create the optimizer

In [6]:
optimizer = torch.optim.Adam(model.parameters(),
                             lr=8e-3,
                             weight_decay=1e-4)
print("---------------------- type of optimizer", type(optimizer))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
print("---------------------- type of scheduler", type(scheduler))

---------------------- type of optimizer <class 'torch.optim.adam.Adam'>
---------------------- type of scheduler <class 'torch.optim.lr_scheduler.CosineAnnealingLR'>
